## Importing Libraries

In [408]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
from numpy.random import randn
import lxml.html as lh

## Scrape the wiki page

In [409]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
doc = lh.fromstring(source.content)
tr_elements = doc.xpath('//tr')

In [410]:
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

### Create empty list

In [411]:
col = []
i = 0

In [412]:
#for each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [413]:
for j in range(1,len(tr_elements)):
    T = tr_elements[j]
    
    if len(T)!=3:
        break
    
    i = 0
    for t in T.iterchildren():
        data=t.text_content()
        if i >0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [414]:
[len(C) for (title,C) in col]

[289, 289, 289]

## Covert the table on wiki to pandas dataframe

In [415]:
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)

In [416]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [417]:
df['Neighbourhood'] = df['Neighbourhood\n']
df.drop('Neighbourhood\n',axis=1,inplace=True)



In [418]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [419]:
## remove"\n"
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x: x.split('\n')[0])

In [420]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [421]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 3 columns):
Postcode         289 non-null object
Borough          289 non-null object
Neighbourhood    289 non-null object
dtypes: object(3)
memory usage: 6.9+ KB


In [422]:
##remove those Borough = "Not assigned"
cdf = df[df['Borough']!='Not assigned']

In [423]:
cdf.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [424]:
cdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 2 to 287
Data columns (total 3 columns):
Postcode         212 non-null object
Borough          212 non-null object
Neighbourhood    212 non-null object
dtypes: object(3)
memory usage: 6.6+ KB


In [425]:
cdf.reset_index(inplace = True)
cdf.drop('index',1,inplace=True)

/Users/derrick7/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [426]:
cdf.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [427]:
cdf.at[6,'Neighbourhood'] = "Queen's Park"

In [428]:
cdf['PostalCode']=cdf['Postcode']


/Users/derrick7/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [429]:
cdf.drop('Postcode',1,inplace=True)
cdf.head()

/Users/derrick7/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Borough,Neighbourhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


In [430]:
## create groupby object of Postalcode and Borough
cdf.set_index(['PostalCode','Borough'],inplace=True)
res = cdf.groupby(level=['PostalCode','Borough'], sort=False).agg( ','.join)
res.dtypes

Neighbourhood    object
dtype: object

In [431]:
res.reset_index(inplace=True)

In [432]:
res

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


## Filter the dataframe to our target data: Toronto and Scarborough

In [433]:
res.columns

Index(['PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

In [434]:
res['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [435]:
tar_data = res[(res['Borough']=='Downtown Toronto') | 
               (res['Borough']=='East Toronto') | 
               (res['Borough']=='West Toronto')|
               (res['Borough']=='Central Toronto')|
              (res['Borough'] == 'Scarborough')]

In [436]:
tar_data.head()

,PostalCode,Borough,Neighbourhood
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M1B,Scarborough,"Rouge,Malvern"
9,M5B,Downtown Toronto,"Ryerson,Garden District"
12,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
15,M5C,Downtown Toronto,St. James Town


In [437]:
tar_data.reset_index(inplace=True)

In [438]:
tar_data.drop('index',1,inplace=True)


/Users/derrick7/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [439]:
tar_data

,PostalCode,Borough,Neighbourhood
0,M5A,Downtown Toronto,"Harbourfront,Regent Park"
1,M1B,Scarborough,"Rouge,Malvern"
2,M5B,Downtown Toronto,"Ryerson,Garden District"
3,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
4,M5C,Downtown Toronto,St. James Town
5,M1E,Scarborough,"Guildwood,Morningside,West Hill"
6,M4E,East Toronto,The Beaches
7,M5E,Downtown Toronto,Berczy Park
8,M1G,Scarborough,Woburn
9,M5G,Downtown Toronto,Central Bay Street


## Segmenting and Clustering the neighbourhoods

In [565]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

url="http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
coordinates=pd.read_csv(url)
coordinates.columns = ['PostalCode', 'Latitude', 'Longitude']
df1 = pd.merge(tar_data,coordinates, on="PostalCode")



Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [566]:
df1.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
3,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [567]:
df1.shape

(55, 5)

## Create Toronto and Scarborough map 

In [568]:
# Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

/Users/derrick7/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [682]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df1['Latitude'], df1['Longitude'], df1['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

## Access Foursquare API

In [570]:
CLIENT_ID = 'B2QVWSGJSIWYJZBWP0G0MNWQKEJBDRUU4J3YDRZW1DCVKVBO' # your Foursquare ID
CLIENT_SECRET = 'WTK4MDXY3LRSCOJHB0KMTJYD4PAVS5P0HDAYQHOFZR5CPNAB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B2QVWSGJSIWYJZBWP0G0MNWQKEJBDRUU4J3YDRZW1DCVKVBO
CLIENT_SECRET:WTK4MDXY3LRSCOJHB0KMTJYD4PAVS5P0HDAYQHOFZR5CPNAB


In [571]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [572]:
LIMIT = 100
radius=500
Toronto_venues = getNearbyVenues(names=df1['Neighbourhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )


Harbourfront,Regent Park
Rouge,Malvern
Ryerson,Garden District
Highland Creek,Rouge Hill,Port Union
St. James Town
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Woburn
Central Bay Street
Christie
Cedarbrae
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
Clairlea,Golden Mile,Oakridge
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Cliffcrest,Cliffside,Scarborough Village West
Studio District
Birch Cliff,Cliffside West
Lawrence Park
Roselawn
Dorset Park,Scarborough Town Centre,Wexford Heights
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
Maryvale,Wexford
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Agincourt
Davisville
Harbord,University of Toronto
Runnymede,

In [573]:
Toronto_venues.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [574]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [575]:
# Check Count of Venues for Each Neighbourhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
Berczy Park,54,54,54,54,54,54
"Birch Cliff,Cliffside West",4,4,4,4,4,4
"Brockton,Exhibition Place,Parkdale Village",20,20,20,20,20,20
Business reply mail Processing Centre969 Eastern,18,18,18,18,18,18
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",47,47,47,47,47,47


In [576]:
# Get the List of Unique Categories
print('There are {} neighbourhoods and {} uniques categories.'.format(Toronto_venues['Neighborhood'].count(),len(Toronto_venues['Venue Category'].unique())))

There are 1800 neighbourhoods and 245 uniques categories.


In [577]:
# one hot encoding
venues_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [578]:
Toronto_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.01
1,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Agincourt North,L'Amoreaux East,Milliken,Steeles East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"Birch Cliff,Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Business reply mail Processing Centre969 Eastern,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",0.000000,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Cedarbrae,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [579]:
Toronto_grouped.shape

(54, 245)

## List frequency of top 5 venues in each neighbourhood

In [580]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0  Coffee Shop          0.06
1  Café                 0.05
2  Steakhouse           0.04
3  Thai Restaurant      0.04
4  American Restaurant  0.04


----Agincourt----
            venue  freq
0  Skating Rink    0.25
1  Breakfast Spot  0.25
2  Clothing Store  0.25
3  Lounge          0.25
4  Yoga Studio     0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0  Playground          0.5 
1  Park                0.5 
2  Nail Salon          0.0 
3  Metro Station       0.0 
4  Mexican Restaurant  0.0 


----Berczy Park----
            venue  freq
0  Coffee Shop     0.07
1  Restaurant      0.06
2  Cocktail Bar    0.06
3  Farmers Market  0.04
4  Café            0.04


----Birch Cliff,Cliffside West----
                   venue  freq
0  College Stadium        0.25
1  General Entertainment  0.25
2  Skating Rink           0.25
3  Café                   0.25
4  Pharmacy               0.00


----Broc

In [581]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Top 10 venues of each neighbourhood

In [582]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Gym,Asian Restaurant,Clothing Store,Bakery,Bar
1,Agincourt,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steeles East",Playground,Park,Women's Store,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
3,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Café,Pub,Steakhouse,Seafood Restaurant,Farmers Market,Italian Restaurant
4,"Birch Cliff,Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [583]:
neighborhoods_venues_sorted.shape

(54, 11)

In [584]:
neighborhoods_venues_sorted['Neighbourhood']=neighborhoods_venues_sorted['Neighborhood']
neighborhoods_venues_sorted.drop('Neighborhood',1,inplace=True)

## KMeans Clustering of Neighbourhoods

In [585]:
# Distribute in 5 Clusters

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ = np.append(kmeans.labels_,[2])
kmeans.labels_

array([2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 4, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2])

In [586]:
#Dataframe to include Clusters

Toronto_merged = df1
Toronto_merged['Cluster Labels'] = kmeans.labels_





In [587]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head(30) # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2,Coffee Shop,Café,Bakery,Park,Pub,Theater,Mexican Restaurant,Breakfast Spot,French Restaurant,Italian Restaurant
1,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Women's Store,Fish & Chips Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Sandwich Place,Restaurant,Ramen Restaurant,Diner,Plaza
3,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2,Bar,Golf Course,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Cocktail Bar,Gastropub,Bakery,Cosmetics Shop,Farmers Market
5,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2,Pizza Place,Rental Car Location,Mexican Restaurant,Electronics Store,Breakfast Spot,Medical Center,Cosmetics Shop,Dog Run,Falafel Restaurant,Event Space
6,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Astrologer,Pub,Coffee Shop,Furniture / Home Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store
7,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Café,Pub,Steakhouse,Seafood Restaurant,Farmers Market,Italian Restaurant
8,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
9,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Bar,Café,Bubble Tea Shop,Japanese Restaurant,Spa


## Map of cluster of Neighbourhoods

In [683]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Clusters of 0-4 cluster Neighbourhood

In [589]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Sandwich Place,Restaurant,Ramen Restaurant,Diner,Plaza
40,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450,0,Coffee Shop,Café,Pizza Place,Sushi Restaurant,Italian Restaurant,Falafel Restaurant,Post Office,Food,Butcher,Bar
45,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West",43.686412,-79.400049,0,Light Rail Station,Coffee Shop,Pub,American Restaurant,Convenience Store,Bagel Shop,Sports Bar,Fried Chicken Joint,Supermarket,Sushi Restaurant


In [590]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,M1T,Scarborough,"Clarks Corners,Sullivan,Tam O'Shanter",43.781638,-79.304302,1,Pizza Place,Chinese Restaurant,Fried Chicken Joint,Italian Restaurant,Rental Car Location,Thai Restaurant,Noodle House,Fast Food Restaurant,Creperie,Comfort Food Restaurant


In [591]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2,Coffee Shop,Café,Bakery,Park,Pub,Theater,Mexican Restaurant,Breakfast Spot,French Restaurant,Italian Restaurant
1,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Women's Store,Fish & Chips Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2,Bar,Golf Course,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Cocktail Bar,Gastropub,Bakery,Cosmetics Shop,Farmers Market
5,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2,Pizza Place,Rental Car Location,Mexican Restaurant,Electronics Store,Breakfast Spot,Medical Center,Cosmetics Shop,Dog Run,Falafel Restaurant,Event Space
6,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Astrologer,Pub,Coffee Shop,Furniture / Home Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store
7,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Café,Pub,Steakhouse,Seafood Restaurant,Farmers Market,Italian Restaurant
8,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
9,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Bar,Café,Bubble Tea Shop,Japanese Restaurant,Spa
10,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Café,Grocery Store,Park,Nightclub,Restaurant,Italian Restaurant,Baby Store,Diner,Coffee Shop,Convenience Store


In [592]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,M4Y,Downtown Toronto,Church and Wellesley,43.66586,-79.38316,3,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Gastropub,Men's Store,Mediterranean Restaurant,Pub


In [593]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.38316,4,Playground,Intersection,Trail,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Electronics Store


## Find which cluster does Central Bay Street belong to and find which neighbourhoods in the cluster are from Scarborough

In [594]:
Toronto_merged[Toronto_merged['Neighbourhood']=='Central Bay Street']

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Bar,Café,Bubble Tea Shop,Japanese Restaurant,Spa


In [595]:
neigh_candidates = Toronto_merged.loc[(Toronto_merged['Cluster Labels'] == 2) & (Toronto_merged['Borough'] =='Scarborough')]

In [596]:
neigh_candidates.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Women's Store,Fish & Chips Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2,Bar,Golf Course,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
5,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2,Pizza Place,Rental Car Location,Mexican Restaurant,Electronics Store,Breakfast Spot,Medical Center,Cosmetics Shop,Dog Run,Falafel Restaurant,Event Space
8,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
11,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Bank,Bakery,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Lounge,Hakka Restaurant,Event Space,Ethiopian Restaurant


### Create map of target neighbourhoods

In [684]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neigh_candidates['Latitude'], neigh_candidates['Longitude'], neigh_candidates['Neighbourhood'], neigh_candidates['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [598]:
neigh_candidates.reset_index(drop=True,inplace=True)

In [599]:
neigh_candidates

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Women's Store,Fish & Chips Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2,Bar,Golf Course,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2,Pizza Place,Rental Car Location,Mexican Restaurant,Electronics Store,Breakfast Spot,Medical Center,Cosmetics Shop,Dog Run,Falafel Restaurant,Event Space
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Bank,Bakery,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Lounge,Hakka Restaurant,Event Space,Ethiopian Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Playground,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,2,Discount Store,Convenience Store,Chinese Restaurant,Train Station,Department Store,Coffee Shop,Falafel Restaurant,Event Space,Diner,Electronics Store
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,2,Bakery,Bus Line,Soccer Field,Intersection,Metro Station,Bus Station,Fast Food Restaurant,Park,Creperie,Doner Restaurant
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,2,Intersection,Motel,American Restaurant,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Diner
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,2,College Stadium,Skating Rink,General Entertainment,Café,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [600]:
neigh_candidates.columns

Index(['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude',
       'Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

## Looking for Chinese restaurants and grocery store

In [601]:
neigh_candidates[neigh_candidates[['1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue']] == 'Grocery Store']

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [602]:
neigh_candidates[neigh_candidates[['1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue']] == 'Chinese Restaurant']

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chinese Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## looking for grocery stores in neighbourhood id 6,10,14

In [603]:
search_query = 'Grocery Store'
radius = 10000
LIMIT = 10
latitude = neigh_candidates.loc[6,'Latitude']
longitude = neigh_candidates.loc[6,'Longitude']
print("latitude is {},longitude is {}".format(latitude,longitude))


latitude is 43.7279292,longitude is -79.26202940000002


### Define the corresponding URL

In [604]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=B2QVWSGJSIWYJZBWP0G0MNWQKEJBDRUU4J3YDRZW1DCVKVBO&client_secret=WTK4MDXY3LRSCOJHB0KMTJYD4PAVS5P0HDAYQHOFZR5CPNAB&ll=43.7279292,-79.26202940000002&v=20180605&query=Grocery Store&radius=10000&limit=10'

### Send the GET Request and examine the results

In [605]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1dc7884c1f677d29133a37'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/conveniencestore_',
       'suffix': '.png'},
      'id': '4d954b0ea243a5684a65b473',
      'name': 'Convenience Store',
      'pluralName': 'Convenience Stores',
      'primary': True,
      'shortName': 'Convenience Store'}],
    'hasPerk': False,
    'id': '4e0921c645ddb2875c54dd07',
    'location': {'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'distance': 9787,
     'formattedAddress': ['Toronto ON', 'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.67436980079757,
       'lng': -79.35847729086593}],
     'lat': 43.67436980079757,
     'lng': -79.35847729086593,
     'state': 'ON'},
    'name': 'Marys Variety And Grocery Store',
    'referralId': 'v-1545455496'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
      

### Get relevant part of JSON and transform it into a pandas dataframe

In [606]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)


In [607]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Marys Variety And Grocery Store,Convenience Store,NaN,CA,Toronto,Canada,NaN,9787,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67436980079757, 'lng': -79.35847729086593}]",43.674370,-79.358477,NaN,ON,4e0921c645ddb2875c54dd07
1,Ghadir Mid-Eastern Grocery,Grocery Store,1821 Lawrence Ave E,CA,Toronto,Canada,Pharmacy Ave,3760,"[1821 Lawrence Ave E (Pharmacy Ave), Toronto ON M1R 2Y3, Canada]","[{'label': 'display', 'lat': 43.743316713534036, 'lng': -79.303645}]",43.743317,-79.303645,M1R 2Y3,ON,4dd19e051838a75196486487
2,The Beer Store,Beer Store,2727 Eglinton Ave E,CA,Scarborough,Canada,at Brimley Rd.,1388,"[2727 Eglinton Ave E (at Brimley Rd.), Scarborough ON M1K 2S2, Canada]","[{'label': 'display', 'lat': 43.7359411, 'lng': -79.2488052}]",43.735941,-79.248805,M1K 2S2,ON,4b2405e2f964a5204b5f24e3
3,The Beer Store,Beer Store,3431 St Clair Ave E,CA,Scarborough,Canada,St Claire and Birchmount,1806,"[3431 St Clair Ave E (St Claire and Birchmount), Scarborough ON M1L 1W6, Canada]","[{'label': 'display', 'lat': 43.7137652, 'lng': -79.2729808}]",43.713765,-79.272981,M1L 1W6,ON,4bd46a1429eb9c748b7e91e1
4,Your Dollar Store With More,Miscellaneous Shop,139 1880 Eglinton Av E,CA,Toronto,Canada,at Pharmacy,2410,"[139 1880 Eglinton Av E (at Pharmacy), Toronto ON M1L 2L1, Canada]","[{'label': 'display', 'lat': 43.72614435647364, 'lng': -79.291894716112}]",43.726144,-79.291895,M1L 2L1,ON,4ec81c3d99115348d9c9ae35
5,The UPS Store,Shipping Store,NaN,CA,Toronto,Canada,NaN,802,"[Toronto ON M5C 2B5, Canada]","[{'label': 'display', 'lat': 43.73503, 'lng': -79.26032}]",43.735030,-79.260320,M5C 2B5,ON,4e5bb04cb0fbdca30a1a69ed
6,Vanessa Flowers & Caribbean Grocery,Convenience Store,118 Anaconda Avenue,CA,Toronto,Canada,Birchmount Road,1375,"[118 Anaconda Avenue (Birchmount Road), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.718455451794945, 'lng': -79.27300945113241}]",43.718455,-79.273009,NaN,ON,4db450424df05e5aaadd1653
7,Red Rose Halal Meat & Grocery,Market,NaN,CA,Toronto,Canada,NaN,1951,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.71132772003987, 'lng': -79.2698110735145}]",43.711328,-79.269811,NaN,ON,53a4499c498ea4bed5aaa128
8,Sherwin-Williams Paint Store,Hardware Store,2050 Eglinton Ave E,CA,Scarborough,Canada,NaN,1603,"[2050 Eglinton Ave E, Scarborough ON M1L 2M6, Canada]","[{'label': 'display', 'lat': 43.72952, 'lng': -79.28184}]",43.729520,-79.281840,M1L 2M6,ON,57cf3083498ec6af01f55ec1
9,Pauls Grocery,Convenience Store,25 Craiglee,CA,NaN,Canada,NaN,2530,"[25 Craiglee, Canada]","[{'label': 'display', 'lat': 43.70546804580729, 'lng': -79.25720366816061}]",43.705468,-79.257204,NaN,NaN,4ef644a4a69dea74f9d785d9


## Clean the data to a table contain only grocery stores

In [619]:
dataframe_filtered = dataframe_filtered[(dataframe_filtered['categories']=="Market") | (dataframe_filtered['categories']==
                                       "Convenience Store") | (dataframe_filtered['categories']=="Grocery Store")]





In [621]:
dataframe_filtered.reset_index(drop=True,inplace=True)

In [622]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Marys Variety And Grocery Store,Convenience Store,NaN,CA,Toronto,Canada,NaN,9787,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67436980079757, 'lng': -79.35847729086593}]",43.674370,-79.358477,NaN,ON,4e0921c645ddb2875c54dd07
1,Ghadir Mid-Eastern Grocery,Grocery Store,1821 Lawrence Ave E,CA,Toronto,Canada,Pharmacy Ave,3760,"[1821 Lawrence Ave E (Pharmacy Ave), Toronto ON M1R 2Y3, Canada]","[{'label': 'display', 'lat': 43.743316713534036, 'lng': -79.303645}]",43.743317,-79.303645,M1R 2Y3,ON,4dd19e051838a75196486487
2,Vanessa Flowers & Caribbean Grocery,Convenience Store,118 Anaconda Avenue,CA,Toronto,Canada,Birchmount Road,1375,"[118 Anaconda Avenue (Birchmount Road), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.718455451794945, 'lng': -79.27300945113241}]",43.718455,-79.273009,NaN,ON,4db450424df05e5aaadd1653
3,Red Rose Halal Meat & Grocery,Market,NaN,CA,Toronto,Canada,NaN,1951,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.71132772003987, 'lng': -79.2698110735145}]",43.711328,-79.269811,NaN,ON,53a4499c498ea4bed5aaa128
4,Pauls Grocery,Convenience Store,25 Craiglee,CA,NaN,Canada,NaN,2530,"[25 Craiglee, Canada]","[{'label': 'display', 'lat': 43.70546804580729, 'lng': -79.25720366816061}]",43.705468,-79.257204,NaN,NaN,4ef644a4a69dea74f9d785d9


In [623]:
CLIENT_ID = 'M5N1UTZY01GY4M4MSEPHTOWRVCO13MYQKH1TORX11FAMGBVI' # your Foursquare ID
CLIENT_SECRET = 'LB3R0HK1QRRCSPHAEDAPJTIDY5S2TYG5TGZEWDTEEO5P0YD5' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M5N1UTZY01GY4M4MSEPHTOWRVCO13MYQKH1TORX11FAMGBVI
CLIENT_SECRET:LB3R0HK1QRRCSPHAEDAPJTIDY5S2TYG5TGZEWDTEEO5P0YD5


## Let's utilize Foursquare to see whether the 5 grocery store are good or not

## Explore the rating of 5 stores

In [627]:
def rate(id):
    venue_id = id
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
    except:
        print('This venue has not been rated yet.')





In [628]:
dataframe_filtered['id'].apply(lambda id: rate(id) )

This venue has not been rated yet.
8.1
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.


0    None
1    None
2    None
3    None
4    None
Name: id, dtype: object

In [630]:
dataframe_filtered.at[1,'rating']=8.1
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id,rating
0,Marys Variety And Grocery Store,Convenience Store,NaN,CA,Toronto,Canada,NaN,9787,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67436980079757, 'lng': -79.35847729086593}]",43.674370,-79.358477,NaN,ON,4e0921c645ddb2875c54dd07,no rating yet
1,Ghadir Mid-Eastern Grocery,Grocery Store,1821 Lawrence Ave E,CA,Toronto,Canada,Pharmacy Ave,3760,"[1821 Lawrence Ave E (Pharmacy Ave), Toronto ON M1R 2Y3, Canada]","[{'label': 'display', 'lat': 43.743316713534036, 'lng': -79.303645}]",43.743317,-79.303645,M1R 2Y3,ON,4dd19e051838a75196486487,8.1
2,Vanessa Flowers & Caribbean Grocery,Convenience Store,118 Anaconda Avenue,CA,Toronto,Canada,Birchmount Road,1375,"[118 Anaconda Avenue (Birchmount Road), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.718455451794945, 'lng': -79.27300945113241}]",43.718455,-79.273009,NaN,ON,4db450424df05e5aaadd1653,no rating yet
3,Red Rose Halal Meat & Grocery,Market,NaN,CA,Toronto,Canada,NaN,1951,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.71132772003987, 'lng': -79.2698110735145}]",43.711328,-79.269811,NaN,ON,53a4499c498ea4bed5aaa128,no rating yet
4,Pauls Grocery,Convenience Store,25 Craiglee,CA,NaN,Canada,NaN,2530,"[25 Craiglee, Canada]","[{'label': 'display', 'lat': 43.70546804580729, 'lng': -79.25720366816061}]",43.705468,-79.257204,NaN,NaN,4ef644a4a69dea74f9d785d9,no rating yet


## It's a good rating, let's get the tips

In [631]:
result['response']['venue']['tips']['count']

2

In [632]:

limit = 2 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5c1ea1e86a607133f3f4bbe2'},
 'response': {'tips': {'count': 2,
   'items': [{'agreeCount': 0,
     'authorInteractionType': 'liked',
     'canonicalUrl': 'https://foursquare.com/item/53388d95498ec9e627846531',
     'createdAt': 1396215189,
     'disagreeCount': 0,
     'id': '53388d95498ec9e627846531',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'text': 'Best Halal butcher shop around!',
     'todo': {'count': 0},
     'type': 'user',
     'user': {'firstName': 'Hashem',
      'gender': 'male',
      'id': '15925066',
      'lastName': 'A',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/HGFG3IW0OFRL3K0D.jpg'}}}]}}}

In [633]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [634]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Best Halal butcher shop around!,0,0,53388d95498ec9e627846531,Hashem,A,male,15925066


## That's a good review 

## Explore the neighbourhood id=10

In [635]:
search_query = 'Grocery Store'
radius = 10000
LIMIT = 10
latitude = neigh_candidates.loc[10,'Latitude']
longitude = neigh_candidates.loc[10,'Longitude']
print("latitude is {},longitude is {}".format(latitude,longitude))


latitude is 43.7574096,longitude is -79.27330400000001


In [636]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=M5N1UTZY01GY4M4MSEPHTOWRVCO13MYQKH1TORX11FAMGBVI&client_secret=LB3R0HK1QRRCSPHAEDAPJTIDY5S2TYG5TGZEWDTEEO5P0YD5&ll=43.7574096,-79.27330400000001&v=20180604&query=Grocery Store&radius=10000&limit=10'

In [637]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1ea3414434b91ebed3f903'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d101951735',
      'name': 'Thrift / Vintage Store',
      'pluralName': 'Thrift / Vintage Stores',
      'primary': True,
      'shortName': 'Thrift / Vintage'}],
    'hasPerk': False,
    'id': '4bfebef9e584c928402b6d25',
    'location': {'address': '1225 Kennedy Rd',
     'cc': 'CA',
     'city': 'Scarborough',
     'country': 'Canada',
     'distance': 295,
     'formattedAddress': ['1225 Kennedy Rd',
      'Scarborough ON M1P 2L4',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.755782105509475,
       'lng': -79.2762077195083}],
     'lat': 43.755782105509475,
     'lng': -79.2762077195083,
     'postalCode': 'M1P 2L4',
     'state': 'ON'},
    'name': 'Salvation Army Thrift Store',
    'referralId': 'v-1545511745'},


In [638]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d101951735', 'name': 'Thrift / Vintage Store', 'pluralName': 'Thrift / Vintage Stores', 'shortName': 'Thrift / Vintage', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_', 'suffix': '.png'}, 'primary': True}]",False,4bfebef9e584c928402b6d25,1225 Kennedy Rd,CA,Scarborough,Canada,NaN,295,"[1225 Kennedy Rd, Scarborough ON M1P 2L4, Canada]","[{'label': 'display', 'lat': 43.755782105509475, 'lng': -79.2762077195083}]",43.755782,-79.276208,NaN,M1P 2L4,ON,Salvation Army Thrift Store,v-1545511745
1,"[{'id': '4bf58dd8d48988d103951735', 'name': 'Clothing Store', 'pluralName': 'Clothing Stores', 'shortName': 'Apparel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_', 'suffix': '.png'}, 'primary': True}]",False,4d1127d65f3376eb86661386,300 Borough Dr,CA,Scarborough,Canada,NaN,2435,"[300 Borough Dr, Scarborough ON M1P 4P5, Canada]","[{'label': 'display', 'lat': 43.77601546103458, 'lng': -79.25736884156035}]",43.776015,-79.257369,Scarborough City Centre,M1P 4P5,ON,Tommy Hilfiger Company Store,v-1545511745
2,"[{'id': '4bf58dd8d48988d1f3941735', 'name': 'Toy / Game Store', 'pluralName': 'Toy / Game Stores', 'shortName': 'Toys & Games', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/toys_', 'suffix': '.png'}, 'primary': True}]",False,5085ec39e4b0b1ead2eb0818,300 Borough Drive,CA,Scarborough,Canada,in Scarborough Town Centre,2413,"[300 Borough Drive (in Scarborough Town Centre), Scarborough ON M1P 4P5, Canada]","[{'label': 'display', 'lat': 43.775537, 'lng': -79.256833}]",43.775537,-79.256833,NaN,M1P 4P5,ON,Disney Store,v-1545511745
3,"[{'id': '4bf58dd8d48988d118951735', 'name': 'Grocery Store', 'pluralName': 'Grocery Stores', 'shortName': 'Grocery Store', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}, 'primary': True}]",False,4dd19e051838a75196486487,1821 Lawrence Ave E,CA,Toronto,Canada,Pharmacy Ave,2900,"[1821 Lawrence Ave E (Pharmacy Ave), Toronto ON M1R 2Y3, Canada]","[{'label': 'display', 'lat': 43.743316713534036, 'lng': -79.303645}]",43.743317,-79.303645,NaN,M1R 2Y3,ON,Ghadir Mid-Eastern Grocery,v-1545511745
4,"[{'id': '4bf58dd8d48988d112951735', 'name': 'Hardware Store', 'pluralName': 'Hardware Stores', 'shortName': 'Hardware', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/hardware_', 'suffix': '.png'}, 'primary': True}]",False,57cf339c498eef8d7dfa0251,260 Nantucket Blvd,CA,Scarborough,Canada,NaN,432,"[260 Nantucket Blvd, Scarborough ON M1P 2P4, Canada]","[{'label': 'display', 'lat': 43.76047864, 'lng': -79.27659548}]",43.760479,-79.276595,NaN,M1P 2P4,ON,Sherwin-Williams Commercial Paint Store,v-1545511745


In [639]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Salvation Army Thrift Store,Thrift / Vintage Store,1225 Kennedy Rd,CA,Scarborough,Canada,NaN,295,"[1225 Kennedy Rd, Scarborough ON M1P 2L4, Canada]","[{'label': 'display', 'lat': 43.755782105509475, 'lng': -79.2762077195083}]",43.755782,-79.276208,NaN,M1P 2L4,ON,4bfebef9e584c928402b6d25
1,Tommy Hilfiger Company Store,Clothing Store,300 Borough Dr,CA,Scarborough,Canada,NaN,2435,"[300 Borough Dr, Scarborough ON M1P 4P5, Canada]","[{'label': 'display', 'lat': 43.77601546103458, 'lng': -79.25736884156035}]",43.776015,-79.257369,Scarborough City Centre,M1P 4P5,ON,4d1127d65f3376eb86661386
2,Disney Store,Toy / Game Store,300 Borough Drive,CA,Scarborough,Canada,in Scarborough Town Centre,2413,"[300 Borough Drive (in Scarborough Town Centre), Scarborough ON M1P 4P5, Canada]","[{'label': 'display', 'lat': 43.775537, 'lng': -79.256833}]",43.775537,-79.256833,NaN,M1P 4P5,ON,5085ec39e4b0b1ead2eb0818
3,Ghadir Mid-Eastern Grocery,Grocery Store,1821 Lawrence Ave E,CA,Toronto,Canada,Pharmacy Ave,2900,"[1821 Lawrence Ave E (Pharmacy Ave), Toronto ON M1R 2Y3, Canada]","[{'label': 'display', 'lat': 43.743316713534036, 'lng': -79.303645}]",43.743317,-79.303645,NaN,M1R 2Y3,ON,4dd19e051838a75196486487
4,Sherwin-Williams Commercial Paint Store,Hardware Store,260 Nantucket Blvd,CA,Scarborough,Canada,NaN,432,"[260 Nantucket Blvd, Scarborough ON M1P 2P4, Canada]","[{'label': 'display', 'lat': 43.76047864, 'lng': -79.27659548}]",43.760479,-79.276595,NaN,M1P 2P4,ON,57cf339c498eef8d7dfa0251
5,Everything for a Dollar Store,Miscellaneous Shop,2650 Lawrence Ave E,CA,Scarborough,Canada,NaN,931,"[2650 Lawrence Ave E, Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.7523719727376, 'lng': -79.2640471121426}]",43.752372,-79.264047,NaN,NaN,ON,4ec96e336c251306cdc4732a
6,Pack and Store,Storage Facility,1399 Kennedy Road,CA,Scarborough,Canada,kennedy and ellesmere,720,"[1399 Kennedy Road (kennedy and ellesmere), Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.762874603271484, 'lng': -79.27810668945312}]",43.762875,-79.278107,NaN,NaN,ON,546ba557498ede4d7a37aa7f
7,spirit Halloween Store,Miscellaneous Shop,1333 Kennedy Road Scarborough,CA,NaN,Canada,NaN,668,"[1333 Kennedy Road Scarborough, Canada]","[{'label': 'display', 'lat': 43.76163979192874, 'lng': -79.2792132215197}]",43.761640,-79.279213,NaN,NaN,NaN,524852cd11d2e58c893f7088
8,Gap Factory Store,Clothing Store,1900 Eglinton Ave E,CA,Scarborough,Canada,Warden Ave.,3541,"[1900 Eglinton Ave E (Warden Ave.), Scarborough ON M1L 2L8, Canada]","[{'label': 'display', 'lat': 43.72895917270709, 'lng': -79.29300192964865}]",43.728959,-79.293002,NaN,M1L 2L8,ON,4ce803a1948f224b52b1ed5d
9,Metro Pharmacy,Pharmacy,15 Ellesmere Road,CA,Scarborough,Canada,at Parkway Mall,3252,"[15 Ellesmere Road (at Parkway Mall), Scarborough ON M1R 4B7, Canada]","[{'label': 'display', 'lat': 43.756817097834244, 'lng': -79.31375170776809}]",43.756817,-79.313752,Wexford,M1R 4B7,ON,4bc603bf6c26b7135125ecf3


In [640]:
dataframe_filtered = dataframe_filtered[(dataframe_filtered['categories']=="Market") | (dataframe_filtered['categories']==
                                       "Convenience Store") | (dataframe_filtered['categories']=="Grocery Store")]





In [641]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
3,Ghadir Mid-Eastern Grocery,Grocery Store,1821 Lawrence Ave E,CA,Toronto,Canada,Pharmacy Ave,2900,"[1821 Lawrence Ave E (Pharmacy Ave), Toronto ON M1R 2Y3, Canada]","[{'label': 'display', 'lat': 43.743316713534036, 'lng': -79.303645}]",43.743317,-79.303645,NaN,M1R 2Y3,ON,4dd19e051838a75196486487


### It turns out no.10 neighbourhood is close to the same grocery store as no.6

In [642]:
search_query = 'Grocery Store'
radius = 10000
LIMIT = 10
latitude = neigh_candidates.loc[14,'Latitude']
longitude = neigh_candidates.loc[14,'Longitude']
print("latitude is {},longitude is {}".format(latitude,longitude))

latitude is 43.799525200000005,longitude is -79.3183887


In [643]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=M5N1UTZY01GY4M4MSEPHTOWRVCO13MYQKH1TORX11FAMGBVI&client_secret=LB3R0HK1QRRCSPHAEDAPJTIDY5S2TYG5TGZEWDTEEO5P0YD5&ll=43.799525200000005,-79.3183887&v=20180604&query=Grocery Store&radius=10000&limit=10'

In [644]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1ea4736a607133fa112860'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'primary': True,
      'shortName': 'Grocery Store'}],
    'hasPerk': False,
    'id': '4cb0801475ebb60c94eac4ad',
    'location': {'address': '170 Esna Park Dr, Unit 8',
     'cc': 'CA',
     'city': 'Markham',
     'country': 'Canada',
     'crossStreet': 'at Esna Park Dr. & Denison St.',
     'distance': 3215,
     'formattedAddress': ['170 Esna Park Dr, Unit 8 (at Esna Park Dr. & Denison St.)',
      'Markham ON L3R 1E3',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.82515878066295,
       'lng': -79.33683765481253}],
     'lat': 43.82515878066295,
     'lng': -79.33683765481253,
     'postalCode': 'L3R 1E3',
     'state': 'ON

In [645]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'Grocery Store', 'pluralName': 'Grocery Stores', 'shortName': 'Grocery Store', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}, 'primary': True}]",False,4cb0801475ebb60c94eac4ad,"170 Esna Park Dr, Unit 8",CA,Markham,Canada,at Esna Park Dr. & Denison St.,3215,"[170 Esna Park Dr, Unit 8 (at Esna Park Dr. & Denison St.), Markham ON L3R 1E3, Canada]","[{'label': 'display', 'lat': 43.82515878066295, 'lng': -79.33683765481253}]",43.825159,-79.336838,NaN,L3R 1E3,ON,The Low Carb Grocery,v-1545512051
1,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'Food & Drink Shop', 'pluralName': 'Food & Drink Shops', 'shortName': 'Food & Drink', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/foodanddrink_', 'suffix': '.png'}, 'primary': True}]",False,4e889087775b68cb83120b25,11 ivy bush ave,CA,Scarborough,Canada,NaN,1356,"[11 ivy bush ave, Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.81012949300771, 'lng': -79.31007182562468}]",43.810129,-79.310072,NaN,NaN,ON,Zain's Indian Grocery,v-1545512051
2,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'Miscellaneous Shop', 'pluralName': 'Miscellaneous Shops', 'shortName': 'Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_', 'suffix': '.png'}, 'primary': True}]",False,4cfc0ac134c1a0931d46490e,NaN,CA,NaN,Canada,NaN,908,[Canada],"[{'label': 'display', 'lat': 43.797679, 'lng': -79.329409}]",43.797679,-79.329409,NaN,NaN,NaN,Quick Look Convenience Store,v-1545512051
3,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'Beer Store', 'pluralName': 'Beer Stores', 'shortName': 'Beer Store', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/beergarden_', 'suffix': '.png'}, 'primary': True}]",False,4be0a48f4f15c9280fedcb0b,2934 Finch Ave. E,CA,Scarborough,Canada,at Victoria Park Ave.,1111,"[2934 Finch Ave. E (at Victoria Park Ave.), Scarborough ON M1W 2T4, Canada]","[{'label': 'display', 'lat': 43.7942512, 'lng': -79.3301331}]",43.794251,-79.330133,NaN,M1W 2T4,ON,The Beer Store,v-1545512051
4,"[{'id': '4bf58dd8d48988d103951735', 'name': 'Clothing Store', 'pluralName': 'Clothing Stores', 'shortName': 'Apparel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_', 'suffix': '.png'}, 'primary': True}]",False,4d1127d65f3376eb86661386,300 Borough Dr,CA,Scarborough,Canada,NaN,5558,"[300 Borough Dr, Scarborough ON M1P 4P5, Canada]","[{'label': 'display', 'lat': 43.77601546103458, 'lng': -79.25736884156035}]",43.776015,-79.257369,Scarborough City Centre,M1P 4P5,ON,Tommy Hilfiger Company Store,v-1545512051


In [646]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Low Carb Grocery,Grocery Store,"170 Esna Park Dr, Unit 8",CA,Markham,Canada,at Esna Park Dr. & Denison St.,3215,"[170 Esna Park Dr, Unit 8 (at Esna Park Dr. & Denison St.), Markham ON L3R 1E3, Canada]","[{'label': 'display', 'lat': 43.82515878066295, 'lng': -79.33683765481253}]",43.825159,-79.336838,NaN,L3R 1E3,ON,4cb0801475ebb60c94eac4ad
1,Zain's Indian Grocery,Food & Drink Shop,11 ivy bush ave,CA,Scarborough,Canada,NaN,1356,"[11 ivy bush ave, Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.81012949300771, 'lng': -79.31007182562468}]",43.810129,-79.310072,NaN,NaN,ON,4e889087775b68cb83120b25
2,Quick Look Convenience Store,Miscellaneous Shop,NaN,CA,NaN,Canada,NaN,908,[Canada],"[{'label': 'display', 'lat': 43.797679, 'lng': -79.329409}]",43.797679,-79.329409,NaN,NaN,NaN,4cfc0ac134c1a0931d46490e
3,The Beer Store,Beer Store,2934 Finch Ave. E,CA,Scarborough,Canada,at Victoria Park Ave.,1111,"[2934 Finch Ave. E (at Victoria Park Ave.), Scarborough ON M1W 2T4, Canada]","[{'label': 'display', 'lat': 43.7942512, 'lng': -79.3301331}]",43.794251,-79.330133,NaN,M1W 2T4,ON,4be0a48f4f15c9280fedcb0b
4,Tommy Hilfiger Company Store,Clothing Store,300 Borough Dr,CA,Scarborough,Canada,NaN,5558,"[300 Borough Dr, Scarborough ON M1P 4P5, Canada]","[{'label': 'display', 'lat': 43.77601546103458, 'lng': -79.25736884156035}]",43.776015,-79.257369,Scarborough City Centre,M1P 4P5,ON,4d1127d65f3376eb86661386
5,Kingdom Christian Book Store,Bookstore,NaN,CA,NaN,Canada,NaN,2469,[Canada],"[{'label': 'display', 'lat': 43.81976896507086, 'lng': -79.33096864739885}]",43.819769,-79.330969,NaN,NaN,NaN,577c097ccd10bd047e763f3f
6,The grocery counter,Snack Place,NaN,CA,Markham,Canada,NaN,7096,"[Markham ON, Canada]","[{'label': 'display', 'lat': 43.85990586365545, 'lng': -79.3467378616333}]",43.859906,-79.346738,NaN,NaN,ON,4bef24ffea570f4774fa8ed2
7,Becker's,Convenience Store,2942 Finch Ave. E.,CA,Toronto,Canada,NaN,1071,"[2942 Finch Ave. E., Toronto ON M1W 2T4, Canada]","[{'label': 'display', 'lat': 43.7943845, 'lng': -79.3296615}]",43.794384,-79.329662,L'Amoreaux,M1W 2T4,ON,4ccb5144ee23a143068a16a8
8,The LEGO Store,Toy / Game Store,1800 Sheppard Ave E,CA,Toronto,Canada,at Don Mills Rd,3114,"[1800 Sheppard Ave E (at Don Mills Rd), Toronto ON M2J 5A7, Canada]","[{'label': 'display', 'lat': 43.77820727238842, 'lng': -79.34348299621146}]",43.778207,-79.343483,NaN,M2J 5A7,ON,4e848fbb5c5c9240de8e6a80
9,Disney Store,Toy / Game Store,300 Borough Drive,CA,Scarborough,Canada,in Scarborough Town Centre,5621,"[300 Borough Drive (in Scarborough Town Centre), Scarborough ON M1P 4P5, Canada]","[{'label': 'display', 'lat': 43.775537, 'lng': -79.256833}]",43.775537,-79.256833,NaN,M1P 4P5,ON,5085ec39e4b0b1ead2eb0818


In [647]:
dataframe_filtered = dataframe_filtered[dataframe_filtered['categories']=='Grocery Store']

In [648]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Low Carb Grocery,Grocery Store,"170 Esna Park Dr, Unit 8",CA,Markham,Canada,at Esna Park Dr. & Denison St.,3215,"[170 Esna Park Dr, Unit 8 (at Esna Park Dr. & Denison St.), Markham ON L3R 1E3, Canada]","[{'label': 'display', 'lat': 43.82515878066295, 'lng': -79.33683765481253}]",43.825159,-79.336838,NaN,L3R 1E3,ON,4cb0801475ebb60c94eac4ad


In [649]:
venue_id = '4cb0801475ebb60c94eac4ad' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4cb0801475ebb60c94eac4ad?client_id=M5N1UTZY01GY4M4MSEPHTOWRVCO13MYQKH1TORX11FAMGBVI&client_secret=LB3R0HK1QRRCSPHAEDAPJTIDY5S2TYG5TGZEWDTEEO5P0YD5&v=20180604'

In [650]:
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.2


### It's not a good rating comparing to Ghadir Mid-Eastern Grocery

In [655]:
final_candidate = neigh_candidates.drop([0,1,2,3,4,5,7,8,9,11,12,13,14,15])

In [656]:
final_candidate

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,2,Discount Store,Convenience Store,Chinese Restaurant,Train Station,Department Store,Coffee Shop,Falafel Restaurant,Event Space,Diner,Electronics Store
10,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford Heights",43.757410,-79.273304,2,Indian Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Ethiopian Restaurant,Electronics Store,Event Space,Eastern European Restaurant


## Let's explore the Chinese Restaurant for no.6 neighbourhood

In [657]:
latitude = final_candidate.loc[6,'Latitude']
longitude = final_candidate.loc[6,'Longitude']
print("latitude is {},longitude is {}".format(latitude,longitude))

search_query = 'Chinese Restaurant'
radius = 10000
print(search_query + ' .... OK!')

latitude is 43.7279292,longitude is -79.26202940000002
Chinese Restaurant .... OK!


In [658]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=M5N1UTZY01GY4M4MSEPHTOWRVCO13MYQKH1TORX11FAMGBVI&client_secret=LB3R0HK1QRRCSPHAEDAPJTIDY5S2TYG5TGZEWDTEEO5P0YD5&ll=43.7279292,-79.26202940000002&v=20180604&query=Chinese Restaurant&radius=10000&limit=10'

In [659]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1ea814f594df03f087eb73'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'primary': True,
      'shortName': 'Chinese'}],
    'hasPerk': False,
    'id': '4bd784e40b779c747a6e05a0',
    'location': {'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'crossStreet': 'Kennedy',
     'distance': 704,
     'formattedAddress': ['Toronto ON', 'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.73329707525865,
       'lng': -79.26666763901558}],
     'lat': 43.73329707525865,
     'lng': -79.26666763901558,
     'state': 'ON'},
    'name': 'Good Luck Chinese Restaurant',
    'referralId': 'v-1545512980'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       

In [660]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4bd784e40b779c747a6e05a0,NaN,CA,Toronto,Canada,Kennedy,704,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.73329707525865, 'lng': -79.26666763901558}]",43.733297,-79.266668,NaN,ON,Good Luck Chinese Restaurant,v-1545512980
1,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4b79de5ff964a52036172fe3,4002 Sheppard Ave. E,CA,Scarborough,Canada,at Kennedy Rd.,6538,"[4002 Sheppard Ave. E (at Kennedy Rd.), Scarborough ON M1S 4R5, Canada]","[{'label': 'display', 'lat': 43.78367044432938, 'lng': -79.28767856291728}]",43.783670,-79.287679,M1S 4R5,ON,Very Fair Chinese Restaurant 同德樓,v-1545512980
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4cdc8e53d4ecb1f7843c8048,3587 Sheppard Ave E,CA,Toronto,Canada,Birchmount Rd,6558,"[3587 Sheppard Ave E (Birchmount Rd), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.78050473445372, 'lng': -79.29884391314476}]",43.780505,-79.298844,NaN,ON,The Royal Chinese Restaurant 避風塘小炒,v-1545512980
3,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4b29e021f964a520d4a324e3,4386 Sheppard Ave. E,CA,Toronto,Canada,at Brimley Rd.,6694,"[4386 Sheppard Ave. E (at Brimley Rd.), Toronto ON M1S 1T8, Canada]","[{'label': 'display', 'lat': 43.78777423062292, 'lng': -79.27029393705004}]",43.787774,-79.270294,M1S 1T8,ON,Perfect Chinese Restaurant 雅瓊海鮮酒家,v-1545512980
4,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4b5cde6af964a5206a4829e3,398 Ferrier St.,CA,Markham,Canada,NaN,11682,"[398 Ferrier St., Markham ON L3R 2Z5, Canada]","[{'label': 'display', 'lat': 43.820017337218836, 'lng': -79.3317460739485}]",43.820017,-79.331746,L3R 2Z5,ON,Century Palace Chinese Restaurant 世紀皇宮大酒樓,v-1545512980


In [661]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Good Luck Chinese Restaurant,Chinese Restaurant,NaN,CA,Toronto,Canada,Kennedy,704,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.73329707525865, 'lng': -79.26666763901558}]",43.733297,-79.266668,NaN,ON,4bd784e40b779c747a6e05a0
1,Very Fair Chinese Restaurant 同德樓,Chinese Restaurant,4002 Sheppard Ave. E,CA,Scarborough,Canada,at Kennedy Rd.,6538,"[4002 Sheppard Ave. E (at Kennedy Rd.), Scarborough ON M1S 4R5, Canada]","[{'label': 'display', 'lat': 43.78367044432938, 'lng': -79.28767856291728}]",43.783670,-79.287679,M1S 4R5,ON,4b79de5ff964a52036172fe3
2,The Royal Chinese Restaurant 避風塘小炒,Chinese Restaurant,3587 Sheppard Ave E,CA,Toronto,Canada,Birchmount Rd,6558,"[3587 Sheppard Ave E (Birchmount Rd), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.78050473445372, 'lng': -79.29884391314476}]",43.780505,-79.298844,NaN,ON,4cdc8e53d4ecb1f7843c8048
3,Perfect Chinese Restaurant 雅瓊海鮮酒家,Chinese Restaurant,4386 Sheppard Ave. E,CA,Toronto,Canada,at Brimley Rd.,6694,"[4386 Sheppard Ave. E (at Brimley Rd.), Toronto ON M1S 1T8, Canada]","[{'label': 'display', 'lat': 43.78777423062292, 'lng': -79.27029393705004}]",43.787774,-79.270294,M1S 1T8,ON,4b29e021f964a520d4a324e3
4,Century Palace Chinese Restaurant 世紀皇宮大酒樓,Chinese Restaurant,398 Ferrier St.,CA,Markham,Canada,NaN,11682,"[398 Ferrier St., Markham ON L3R 2Z5, Canada]","[{'label': 'display', 'lat': 43.820017337218836, 'lng': -79.3317460739485}]",43.820017,-79.331746,L3R 2Z5,ON,4b5cde6af964a5206a4829e3
5,Lin Chinese Restaurant,Chinese Restaurant,NaN,CA,Toronto,Canada,Birchmount and Eglinton,1386,"[Birchmount and Eglinton, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.735107, 'lng': -79.276122}]",43.735107,-79.276122,NaN,ON,4b88b912f964a520990a32e3
6,South Sea Fish Village Chinese Restaurant,Chinese Restaurant,1 Glen Watford Dr.,CA,Scarborough,Canada,NaN,6580,"[1 Glen Watford Dr., Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.78620976205095, 'lng': -79.27570140306393}]",43.786210,-79.275701,NaN,ON,4baa46def964a520485a3ae3
7,Best Friends Chinese Restaurant 會賓樓,Chinese Restaurant,4810 Sheppard Ave. E,CA,Scarborough,Canada,at Shorting Rd.,7174,"[4810 Sheppard Ave. E (at Shorting Rd.), Scarborough ON M1S 4N6, Canada]","[{'label': 'display', 'lat': 43.791806, 'lng': -79.250197}]",43.791806,-79.250197,M1S 4N6,ON,4b5738d3f964a520c62b28e3
8,Chopsticks SzeChuan Chinese Restaurant,Chinese Restaurant,1163 Ellesmere Rd.,CA,Scarborough,Canada,at Midland Ave.,4460,"[1163 Ellesmere Rd. (at Midland Ave.), Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.76751369542526, 'lng': -79.27063132190607}]",43.767514,-79.270631,NaN,ON,4c3f8197e26920a17f8a5be7
9,Konner Chinese Restaurant 玉樓東,Chinese Restaurant,126-3250 Midland Ave,CA,Toronto,Canada,at Finch Ave. E,8834,"[126-3250 Midland Ave (at Finch Ave. E), Toronto ON M1V 4W9, Canada]","[{'label': 'display', 'lat': 43.80511163265573, 'lng': -79.2876161018525}]",43.805112,-79.287616,M1V 4W9,ON,4bd3364fa8b3a593955e695f


In [662]:
dataframe_filtered['id'].apply(lambda id: rate(id) )

This venue has not been rated yet.
5.3
7.5
6.0
6.1
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
7.5
This venue has not been rated yet.


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
Name: id, dtype: object

In [664]:
average_rating_no6 = (5.3+7.5+6+6.1+7.5)/5
print("East Birchmount Park,Ionview,Kennedy Park has a average chinese restaurant rating of {}".format(average_rating_no6))   

East Birchmount Park,Ionview,Kennedy Park has a average chinese restaurant rating of 6.4799999999999995


## Let's explore the Chinese Restaurant for no.10 neighbourhood

In [665]:
latitude = final_candidate.loc[10,'Latitude']
longitude = final_candidate.loc[10,'Longitude']
print("latitude is {},longitude is {}".format(latitude,longitude))

search_query = 'Chinese Restaurant'
radius = 10000
print(search_query + ' .... OK!')

latitude is 43.7574096,longitude is -79.27330400000001
Chinese Restaurant .... OK!


In [666]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=M5N1UTZY01GY4M4MSEPHTOWRVCO13MYQKH1TORX11FAMGBVI&client_secret=LB3R0HK1QRRCSPHAEDAPJTIDY5S2TYG5TGZEWDTEEO5P0YD5&ll=43.7574096,-79.27330400000001&v=20180604&query=Chinese Restaurant&radius=10000&limit=10'

In [667]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1ea9654c1f677d449f0549'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'primary': True,
      'shortName': 'Chinese'}],
    'hasPerk': False,
    'id': '4b79de5ff964a52036172fe3',
    'location': {'address': '4002 Sheppard Ave. E',
     'cc': 'CA',
     'city': 'Scarborough',
     'country': 'Canada',
     'crossStreet': 'at Kennedy Rd.',
     'distance': 3143,
     'formattedAddress': ['4002 Sheppard Ave. E (at Kennedy Rd.)',
      'Scarborough ON M1S 4R5',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.78367044432938,
       'lng': -79.28767856291728}],
     'lat': 43.78367044432938,
     'lng': -79.28767856291728,
     'postalCode': 'M1S 4R5',
     'state': 'ON'},
    'name': 'Very Fair Chinese R

In [668]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4b79de5ff964a52036172fe3,4002 Sheppard Ave. E,CA,Scarborough,Canada,at Kennedy Rd.,3143,"[4002 Sheppard Ave. E (at Kennedy Rd.), Scarborough ON M1S 4R5, Canada]","[{'label': 'display', 'lat': 43.78367044432938, 'lng': -79.28767856291728}]",43.783670,-79.287679,M1S 4R5,ON,Very Fair Chinese Restaurant 同德樓,v-1545513317
1,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4cdc8e53d4ecb1f7843c8048,3587 Sheppard Ave E,CA,Toronto,Canada,Birchmount Rd,3290,"[3587 Sheppard Ave E (Birchmount Rd), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.78050473445372, 'lng': -79.29884391314476}]",43.780505,-79.298844,NaN,ON,The Royal Chinese Restaurant 避風塘小炒,v-1545513317
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4c3f8197e26920a17f8a5be7,1163 Ellesmere Rd.,CA,Scarborough,Canada,at Midland Ave.,1145,"[1163 Ellesmere Rd. (at Midland Ave.), Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.76751369542526, 'lng': -79.27063132190607}]",43.767514,-79.270631,NaN,ON,Chopsticks SzeChuan Chinese Restaurant,v-1545513317
3,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4b29e021f964a520d4a324e3,4386 Sheppard Ave. E,CA,Toronto,Canada,at Brimley Rd.,3388,"[4386 Sheppard Ave. E (at Brimley Rd.), Toronto ON M1S 1T8, Canada]","[{'label': 'display', 'lat': 43.78777423062292, 'lng': -79.27029393705004}]",43.787774,-79.270294,M1S 1T8,ON,Perfect Chinese Restaurant 雅瓊海鮮酒家,v-1545513317
4,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4b5cde6af964a5206a4829e3,398 Ferrier St.,CA,Markham,Canada,NaN,8404,"[398 Ferrier St., Markham ON L3R 2Z5, Canada]","[{'label': 'display', 'lat': 43.820017337218836, 'lng': -79.3317460739485}]",43.820017,-79.331746,L3R 2Z5,ON,Century Palace Chinese Restaurant 世紀皇宮大酒樓,v-1545513317


In [669]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Very Fair Chinese Restaurant 同德樓,Chinese Restaurant,4002 Sheppard Ave. E,CA,Scarborough,Canada,at Kennedy Rd.,3143,"[4002 Sheppard Ave. E (at Kennedy Rd.), Scarborough ON M1S 4R5, Canada]","[{'label': 'display', 'lat': 43.78367044432938, 'lng': -79.28767856291728}]",43.783670,-79.287679,M1S 4R5,ON,4b79de5ff964a52036172fe3
1,The Royal Chinese Restaurant 避風塘小炒,Chinese Restaurant,3587 Sheppard Ave E,CA,Toronto,Canada,Birchmount Rd,3290,"[3587 Sheppard Ave E (Birchmount Rd), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.78050473445372, 'lng': -79.29884391314476}]",43.780505,-79.298844,NaN,ON,4cdc8e53d4ecb1f7843c8048
2,Chopsticks SzeChuan Chinese Restaurant,Chinese Restaurant,1163 Ellesmere Rd.,CA,Scarborough,Canada,at Midland Ave.,1145,"[1163 Ellesmere Rd. (at Midland Ave.), Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.76751369542526, 'lng': -79.27063132190607}]",43.767514,-79.270631,NaN,ON,4c3f8197e26920a17f8a5be7
3,Perfect Chinese Restaurant 雅瓊海鮮酒家,Chinese Restaurant,4386 Sheppard Ave. E,CA,Toronto,Canada,at Brimley Rd.,3388,"[4386 Sheppard Ave. E (at Brimley Rd.), Toronto ON M1S 1T8, Canada]","[{'label': 'display', 'lat': 43.78777423062292, 'lng': -79.27029393705004}]",43.787774,-79.270294,M1S 1T8,ON,4b29e021f964a520d4a324e3
4,Century Palace Chinese Restaurant 世紀皇宮大酒樓,Chinese Restaurant,398 Ferrier St.,CA,Markham,Canada,NaN,8404,"[398 Ferrier St., Markham ON L3R 2Z5, Canada]","[{'label': 'display', 'lat': 43.820017337218836, 'lng': -79.3317460739485}]",43.820017,-79.331746,L3R 2Z5,ON,4b5cde6af964a5206a4829e3
5,Kim Ling Indian Chinese Restaurant,Indian Chinese Restaurant,2173 Lawrence Ave. East,CA,Toronto,Canada,at Birchmount Rd,1407,"[2173 Lawrence Ave. East (at Birchmount Rd), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.747400944939095, 'lng': -79.28400200879074}]",43.747401,-79.284002,NaN,ON,4bdb095ec79cc92877c980e9
6,South Sea Fish Village Chinese Restaurant,Chinese Restaurant,1 Glen Watford Dr.,CA,Scarborough,Canada,NaN,3211,"[1 Glen Watford Dr., Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.78620976205095, 'lng': -79.27570140306393}]",43.786210,-79.275701,NaN,ON,4baa46def964a520485a3ae3
7,Best Friends Chinese Restaurant 會賓樓,Chinese Restaurant,4810 Sheppard Ave. E,CA,Scarborough,Canada,at Shorting Rd.,4255,"[4810 Sheppard Ave. E (at Shorting Rd.), Scarborough ON M1S 4N6, Canada]","[{'label': 'display', 'lat': 43.791806, 'lng': -79.250197}]",43.791806,-79.250197,M1S 4N6,ON,4b5738d3f964a520c62b28e3
8,Konner Chinese Restaurant 玉樓東,Chinese Restaurant,126-3250 Midland Ave,CA,Toronto,Canada,at Finch Ave. E,5433,"[126-3250 Midland Ave (at Finch Ave. E), Toronto ON M1V 4W9, Canada]","[{'label': 'display', 'lat': 43.80511163265573, 'lng': -79.2876161018525}]",43.805112,-79.287616,M1V 4W9,ON,4bd3364fa8b3a593955e695f
9,Golden House Chinese Restaurant,Chinese Restaurant,257 Ellesmere Rd,CA,Toronto,Canada,near Warden Ave,2329,"[257 Ellesmere Rd (near Warden Ave), Toronto ON M1R 4E4, Canada]","[{'label': 'display', 'lat': 43.760553, 'lng': -79.30194999488842}]",43.760553,-79.301950,M1R 4E4,ON,4bd879f0f645c9b6e107a8e0


In [670]:
dataframe_filtered['id'].apply(lambda id: rate(id) )

5.3
7.5
7.5
6.0
6.1
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
Name: id, dtype: object

## The ratings of these restaurant are the same of the other candidate, check the map of the two neighbourhoods

In [671]:
address = 'Scarborough, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/Users/derrick7/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


43.773077 -79.257774


In [685]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around scarborough


# add the neighbourhoods as blue circle markers
for lat, lng, label in zip(final_candidate.Latitude, final_candidate.Longitude, final_candidate.Neighbourhood):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Theses two neighbourhoods are our advice to the old couple